Finally, we can do some useful work when mining crypto! The paper [Proofs of Useful Work from Arbitrary Matrix Multiplication](https://arxiv.org/abs/2504.09971) shows that instead of mining Bitcoin by checking hashes, a fundamentally useless operation, we can instead use matrix matrix multiplication, a very useful operation, to mine for BitCoin. This means that when we run AI algorithms, which have tons of matrix multiplications, we can mine Bitcoin for free! Super cool.

<div style="text-align:center">
    <img src="assets/paper.png" alt="logo" width="300">
</div>

The main algorithm has two core components: `solve` and `verify`. The solver can run `solve` and tries to convince the verifier that they did a moderate amount of work. The verifier runs `verify` to check that the solver actually did moderate work. We want to find a task that is difficult enough that it requires moderate work but simple enough that it is easy to check the solution. Running `solve` should be computationally expensive but running `verify` should be computationally cheap. Crucially, if there is a malicious solver who does not actually want to do this computationally expensive work, they w.h.p will not be able to fool the verifier.

In traditional bitcoin PoW, the task is to find a number whose hash has $s$ leading zeros. This is a useless computation. `solve` does a brute force search over all strings, hashes each one, and returns any strings whose hash has $s$ leading zeros. `verify` simply takes the string returned by the solver and runs it through the hash function to confirm that indeed it has $s$ leading zeros. `solve` is computationally expensive because it does brute force search but `verify` is computationally cheap because it just runs a hash function once.

In this paper's PoUW, the task is to multiply two matrices. `solve` adds noise to each matrix, multiplies them, records the mat-mul transcript, hashes the transcript, denoises the mat-mul, and finally returns both the hash and denoised mat-mul. `verify` does the same exact computation as the solver and confirm that it matches both the hash and denoised mat-mul of `solve`. Here `solve` and `verify` are both computationally expensive even though we need `verify` to be computationally cheap. The paper authors note that we can use SNARKs to speed up `verify` (remark 2.2) but the focus of this paper is on the usefulness of `solve`. This seems like a big detail that they skip over...

You might say: if we are simply adding and removing noise to the mat-mul, what's the point? Can't an malicious solver simply skip the step of adding noise? No because they would have a different hashed mat-mul; remember we hash the noisy mat-mul and only remove the noise afterwards. Plus, even if the malicious solver would skip adding the noise, they would still need to multiply matricies and do the same computationally expensive work. So what's the point of the noise? The noise ensures that every time we run a mat-mul on two fixed matrices $A,B$, the result is different since we adding different random noise via a different random seed. This way, a malicious solver cannot precompute the result of $AB$, store it, and then return it anytime we multiply $A,B$ much faster than an honest solver.

Let's take a closer look at the PoUW algorithm:

<div style="text-align:center">
    <img src="assets/solve_verify_alg.png" alt="logo" width="600">
</div>

In `solve` we take in a random seed `sigma`, and two nxn matrices `A,B` defined over the field $\mathbb{F}_q$. We add noise to `A,B` in `encode` and we remove that noise in`decode`. The `matmul` step outputs `C`, the result of multiplying the two noisy matrices, and `z`, the `(n/r)^3` intermediate `rxr`block matrices used when computing the matmul known as the matmul transcript. Although it is not specified here, `z` is really the *hash* of all the `(n/r)^3` intermediate matrix blocks of the transcript.

Let's take a closer look at the `encode` algorithm.

<div style="text-align:center">
    <img src="assets/encode_alg.png" alt="encode" width="600">
</div>

`encode` takes in a random seed `sigma` and the `nxn` matrices `A,B`. 

* Step 1: from the seed, we generate four uniformly pseduorandom matrices `E_L, E_R, F_L, F_R`. The algorithm uses the word "parse" but you can think of this as using a random oracle `O` to deterministically generate `E_L, E_R, F_L, F_R` from the seed `sigma`. This means that the same seed `sigma` produces the same matrices every time but if we don't know sigma, the matrices appear uniformly random. (This is why this algorithm is pseudorandom instead of plain old random.) Mathematically for `E_L`, $P[E_L[i, j] == x] = 1/(2q+1)$ but $P[E_L[i, j] == x | \sigma] = 1 \text{ if } E_L[i, j] == x \text{ else } 0$.
* Step 2: we then multiply the `nxr` and `rxn` low-rank matrices to produce a full-rank `nxn` matrix of uniformly pseudorandom noise. We do this to construct both `E, F`, matrices of pure noise.
* Step 3: Finally, we noise `A,B` by adding our purse noise matrices `E,F` to them.

What is the time complexity? 
* Step 1 takes time $O(nr)$ because we are generating 4 matrices each with a total of `nr` elements. 
* Step 2 takes $cdot O(n^2r)$ because we multiply an `nxr` by `rxn` matrix twice.
* Step 3 takes $O(n^2)$ time because we add two `nxn` matrices twice.
* This yields a total complexity of $O(n^2 r)$ which is bottlenecked by running matmul twice.

Let's take a closer look at the `decode` algorithm.

<div style="text-align:center">
    <img src="assets/decode_alg.png" alt="decode" width="600">
</div>

`decode` takes in a random seed `sigma` and the denoised matmul result `C'`. 

* Step 1: Using the seed `sigma`, we can again get the same uniformly pseudorandom matrices we used in `encode`, `E_L, E_R, F_L, F_R`.
* Step 2: We create `C''`, the inverse of `C'`.
* Step 3: We subtract `C''` from `C'` to remove the noise `E,F` we originally added to `A,B`. This should leave us with the matmul result that has no noise whatsoever.

What is the time complexity?
* Step 1 takes time $O(nr)$ because we are generating 4 matrices each with a total of `nr` elements. 
* Step 2 takes time $O(n^2)$ because we multiply `nxr` and `rxn` matrices. In total we perform five mat-muls here. But we are careful to only multiply `nxr` and `rxn` matrices, never `nxn` matrices.
* Step 3 takes `O(n^2)` for subtracting two `nxn` matrices from each other.

Let's take a closer look at the `MatMul_r` algorithm.

<div style="text-align:center">
    <img src="assets/matmul_alg.png" alt="MatMul algorithm" width="600">
</div>

### Steps

`MatMul_r` takes in an `n×k` matrix `A`, a `k×m` matrix `B`, and a block-size parameter `r`, and outputs an `n×m` matrix `C`. For simplicity, we assume our matrices are all square with `k=m=n`.

**Step 1:** Initialize the `n×m` output matrix `C^(0)` as all zeros.

**Step 2:** Partition `A,B,C` into `rxr` blocks. This creates a grid of `(n/r) × (k/r)` blocks for `A`, `(k/r) × (m/r)` blocks for `B`, and `(n/r) × (m/r)` blocks for `C`.

**Step 3:** Compute the matrix product `C = A·B` using incremental block-wise multiplication $C_{i,j}^{(\ell)} = C_{i,j}^{(\ell-1)} + A_{i,\ell} \cdot B_{\ell,j}$.

**Step 4:** Output the final result.

### Math

Let's dive into the math a bit. The key equation is:

$$
C_{i,j}^{(\ell)} = C_{i,j}^{(\ell-1)} + A_{i,\ell} \cdot B_{\ell,j}
$$

for $i \in [n/r], j \in [m/r], \ell \in [k/r]$, where $C_{i,j}^{(\ell)}$ represents the `(i,j)` block of `C` after `ℓ` iterations.

**Normal matrix multiplication** (entry-by-entry):
- Indices `i, j, k` refer to individual matrix entries
- Each output entry: $C[i,j] = \sum_{k=1}^{n} A[i,k] \cdot B[k,j]$

**Block matrix multiplication** (block-by-block):
- Indices `i, j, ℓ` refer to `r×r` **blocks**, not individual entries
- Each output block computed incrementally:
  - $C_{i,j}^{(1)} = A_{i,1} \cdot B_{1,j}$ After iteration `ℓ=1` we've accumulated contributions from the first `r` terms of the dot product for all entries in the `i,j`-th block matrix
  - $C_{i,j}^{(2)} = C_{i,j}^{(1)} + A_{i,2} \cdot B_{2,j}$ After iteration `ℓ=2` we've accumulated contributions from the first `2r` terms of the dot product for all entries in the `i,j`-th block matrix
  - $C_{i,j}^{(3)} = C_{i,j}^{(2)} + A_{i,3} \cdot B_{3,j}$ After iteration `ℓ=3` we've accumulated contributions from the first `3r` terms of the dot product for all entries in the `i,j`-th block matrix
  - ⋮
  - $C_{i,j}^{(n/r)} = C_{i,j}^{(n/r-1)} + A_{i,n/r} \cdot B_{n/r,j}$ After iteration `ℓ=n/r` we've accumulated contributions from all the `n` terms of the dot product for all entries in the `i,j`-th block matrix.

where $A_{i,\ell}$, $B_{\ell,j}$, and $C_{i,j}^{(\ell)}$ are all `r×r` matrices.

### Complexity 

What is the time complexity?
* Step 1 takes $O(nm)$ because `C` is an `nxm` matrix.
* Step 2 is more of a conceptual step and requires no computation as we can just change our indexing strategy to divide `A,B,C` into `rxr` blocks.
* Step 3 iterates over `n/r` row-block positions (index `i`), `m/r` column-block positions (index `j`), and `k/r` iteration steps (index `ℓ`) for a total of `(n/r) × (m/r) × (k/r) = (n/r)³` iterations for square matrices. Each iteration computes $C_{i,j}^{(\ell)} = C_{i,j}^{(\ell-1)} + A_{i,\ell} \cdot B_{\ell,j}$ which costs $O(r^3)$ time using the naive matmul algorithm to multiply two `rxr` matrices $A_{i,\ell}, B_{\ell,j}$ plus the $O(r^2)$ cost of adding two `rxr` matrices. The total runtime is therefore ($(n/r)^3$ iterations) x ($O(r^3)$ cost per iteration) = $O(n^3)$.
* Step 4 is $O(1)$ since it likely returns a pointer to the resultant matrix `C`.
* In total, this algorithm costs $O(n^3)$ time.

### Algorithm Corrections

This algorithm is actually written imprecisely. First, it says we return $C^{(n)}$ but really the last iteration of $C$ occurs when $\ell = n/r$, $\ell$ does not even reach $n$ so we really return $C^{(n/r)}$. Second, we really want to return the hashed transcript of the matmul, not just the final output of the matmul.

### Hashing

Remark 2.1 states that instead of storing the entire transcript in order to hash it later, we can hash each of the (n/r)³ intermediate matrices as we encounter them and store just the hash, not the transcript itself."

# Code

Let's code up the algorithms `encode`, `decode`, `matmul`, `solve`, and `verify`.

In [1]:
import numpy as np

In [2]:
def _low_rank_noise(n, m, q, r, sigma):
    rng = np.random.default_rng(sigma)
    El = rng.uniform(-q, q, (n, r))
    Er = rng.uniform(-q, q, (r, m))
    Fl = rng.uniform(-q, q, (n, r))
    Fr = rng.uniform(-q, q, (r, m))
    return El, Er, Fl, Fr

In [3]:
def encode(sigma, A, B, r, q):
    # Step 1
    n, m = A.shape[0], B.shape[1]
    El, Er, Fl, Fr = _low_rank_noise(n, m,  q, r, sigma)

    # Step 2
    E = El @ Er
    F = Fl @ Fr

    # Step 3
    A_noise = A + E
    B_noise = B + F

    return A_noise, B_noise

In [4]:
def decode(sigma, A, B, C_noise, r, q):

    # Step 1
    n, m = A.shape[0], B.shape[1]
    El, Er, Fl, Fr = _low_rank_noise(n, m, q, r, sigma)

    # Step 2
    C_noise_inverse = A @ Fl @ Fr + El @ Er @ (B + Fl @ Fr)

    # Step 3
    C = C_noise - C_noise_inverse
    return C

In [18]:
def matmul(A, B, r):

    # Step 1
    n, k, m = A.shape[0], A.shape[1], B.shape[1]
    C = np.zeros((n, m)) # this is really noisy C
    hashes = {}

    for l in range(1, n // r + 1):
        for i in range(n // r): # ith block row
            for j in range(m // r): # jth block column

                # Step 2
                A_block = A[i*r:(i+1)*r, l*r:(l+1)*r]
                B_block = B[l*r:(l+1)*r, j*r:(j+1)*r]
                C_block_prev = C[i*r:(i+1)*r, j*r:(j+1)*r]

                # Step 3
                C_block = C_block_prev + A_block @ B_block
                C[i*r:(i+1)*r, j*r:(j+1)*r] = C_block

                # Hash the intermediate l-th block state
                hashes[(i, j, l)] = hash(C_block.tobytes())

    # Step 4
    return C, hashes

In [19]:
def solve(sigma, A, B, r, q):
    A_noise, B_noise = encode(sigma, A, B, r, q)
    C_noise, hashes = matmul(A_noise, B_noise, r)
    C = decode(sigma, A, B, C_noise, r, q)
    proof = (A, B, hashes)
    return C, proof

In [20]:
def verify(sigma, proof, r, q):
    A, B, solver_hashes = proof

    # the underscore means the verifier computes this, not the solver
    A_noise, B_noise = encode(sigma, A, B, r, q)
    _, verifier_hashes = matmul(A_noise, B_noise, r)

    return solver_hashes == verifier_hashes

Let's confirm this works!

In [21]:
n = 4
r = 1
q = 2
sigma = 42

# 16x16 integer matrices
A = np.arange(n*n).reshape(n, n)
B = np.arange(n*n).reshape(n, n)

A

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [22]:
El, Er, Fl, Fr = _low_rank_noise(n, n, q, r, sigma)
El

array([[ 1.09582419],
       [-0.24448624],
       [ 1.43439168],
       [ 0.78947212]])

In [23]:
C, proof = solve(sigma, A, B, r, q)
C

array([[ 54.47726996,  60.362594  ,  72.1566106 ,  82.22354554],
       [155.76382652, 178.04727824, 185.72584711, 197.67335112],
       [252.85499405, 291.220621  , 310.74727677, 335.78047078],
       [353.17525275, 407.86623602, 426.95422569, 456.44879456]])

In [24]:
hashes = proof[-1]
for i, (k, v) in enumerate(hashes.items()):
    print(k, v)
    if i >= 5:
        break

(0, 0, 1) 959115215560911177
(0, 1, 1) -6345059038964853663
(0, 2, 1) 3086096880040650956
(0, 3, 1) 8986767763871355705
(1, 0, 1) -3973625406403518917
(1, 1, 1) -103071798878682043


In [25]:
verification = verify(sigma, proof, r, q)
verification

True

# Questions

1. So we can technically use any projection matrix to add noise, right? They all satisfy the property that AA^T=I such that the entire computation has no noise but the intermediate computation does have noise added because we have not yet summed up all the terms of the dot product in the transcript, right? But the advantage of using specifically the Hadamard transform as our projection matrix is that it allows us to compute the matmul faster in $O(n^2 \log n)$ time via FFT instead of $O(n^2)$ time, right? Theoretically, it is faster to perform Hadamard matmul with FFT but in practice is it faster? To clarify, when we multiply with the Hadamard matrix, it just means that we can add noise much faster than a regular mat mul but of course we still must compute the regular matmul in time O(n^3), there is no speedup for this.
2. Most LLM inference kernels are memory bound, not compute bound. What's the best tiling parameter `r` for the hardware? Does it align with tensor cores? If we now are computing hashes, does the make us more compute bound or less?